<a href="https://colab.research.google.com/github/baldpanda/advent-of-haystack-2023/blob/main/day_2/advent_of_haystack_day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack - Day 2
_Make a copy of this Colab to start!_


In this challenge, you will complete a scavenger hunt prepared by the Haystack elves. Find what their favorite animals is!!

- [`LinkContentFetcher`](https://docs.haystack.deepset.ai/v2.0/docs/linkcontentfetcher): This will allow you to fetch the contents of https://haystack.deepset.ai/advent-of-haystack/day-1#challenge
- [`HTMLToDocument`](https://docs.haystack.deepset.ai/v2.0/docs/htmltodocument): Once you've fetched the contents, this component will allow you to convert it to a Document
- [`DocumentSplitter`](https://docs.haystack.deepset.ai/v2.0/docs/documentsplitter) (Optional): This is useful if you want to split your Document into chunks
- [`TransformersSimilarityRanker`](https://docs.haystack.deepset.ai/v2.0/docs/transformerssimilarityranker) (Optional): This is useful if you want to rank your documents (chunked with the splitter above) so that the most relevant is at the top.
- [`PromptBuilder`](https://docs.haystack.deepset.ai/v2.0/docs/promptbuilder): This is used to define how you want to prompr an LLM so that it generates an accurate response for you. We’ve included one for you in the starter Colab that will help you with this challenge.
- [`GPTGenerator`](https://docs.haystack.deepset.ai/v2.0/docs/gptgenerator): This component is used to query GPT. You can change this to one of our other generators instead!

#Installation
**Note:** There is a known issue with colab due to a version conflict error related to `llmx` which comes with Colab. You might get an `llmx` error. You can safely ignore this, or run `pip uninstall -y llmx`

In [ ]:
%%bash
pip install haystack-ai
pip install boilerpy3
pip install transformers[torch,sentencepiece]==4.32.1 sentence-transformers>=2.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 12.7 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


Run this code and you’ll be prompted to enter your openAI credentials. If you don’t have a key, [follow these instructions](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key).

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

## Create a Pipeline to fetch the contents of a webpage

Complete the code cell below to create a pipeline that fetches the contents from  https://haystack.deepset.ai/advent-of-haystack/day-1#challenge

In [ ]:
from haystack.components.fetchers.link_content import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.generators import GPTGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack import Pipeline


prompt_template = """
According to these documents:

{% for doc in documents %}
  {{ doc.content }}
{% endfor %}

Answer the given question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(template=prompt_template)

pipeline = Pipeline()

In [ ]:
llm = GPTGenerator(api_key = openai_api_key, model_name = "gpt-4")
fetcher = LinkContentFetcher()
converter = HTMLToDocument()
splitter = DocumentSplitter(split_length=100, split_overlap=5)

In [ ]:
pipeline.add_component(name="fetcher", instance=fetcher)
pipeline.add_component(name="converter", instance=converter)
pipeline.add_component(name="splitter", instance=splitter)
pipeline.add_component(name="prompt_builder", instance=prompt_builder)
pipeline.add_component(name="llm", instance=llm)

In [ ]:
pipeline.connect("fetcher", "converter")
pipeline.connect("converter", "splitter")
pipeline.connect("splitter", "prompt_builder")
pipeline.connect("prompt_builder", "llm")

## Try to find the answer to the question



In [ ]:
question = "What is our favorite animal?"
result = pipeline.run({"prompt_builder": {"question": question}, "fetcher": {"urls": ["https://haystack.deepset.ai/advent-of-haystack/day-1#challenge"]}})

print(result['llm']['replies'][0])

The favorite animal is a capybara.
